In [1]:
import os

os.chdir("../")

HOME = os.getcwd()
print(f"Current working directory: {HOME}")

Current working directory: D:\Crude_Oil_Price_Prediction____Color Sketch\Crude_Oil_Price_Prediction\crude_oil_price_prediction


#### Update entity 

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

##### Update the params.yaml

#### Update the ConfigurationManager

mlProject/config/configuration.py

In [5]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

# from mlProject.entity.config_entity import ModelTrainerConfig

class ConfigurationManager:

    def __init__(
        self,
        config_file_path=CONFIG_FILE_PATH,
        params_file_path=PARAMS_FILE_PATH,
        schema_file_path=SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
        )
        return model_trainer_config

#### Update the components

mlProject/components/model_trainer.py

In [43]:
import os
import pandas as pd
from sklearn.linear_model import ElasticNet
from mlProject import logger
import joblib

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        # Define the training and test csv
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # Drop the Target column from train and test csv data
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)

        # Assign the target
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        # Train
        lr = ElasticNet(alpha=self.config.alpha,
                        l1_ratio=self.config.l1_ratio,
                        random_state=42 )
        lr.fit(train_x, train_y)

        # Save trained model
        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))
        logger.info(f"")

#### Update the Pipeline

mlProject/pipeline/stage_04_model_trainer.py

In [44]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    logger.exception(e)
    raise e

[2024-04-10 22:26:02,668: INFO: common: yaml file: config\config.yaml loaded successfully.]
[2024-04-10 22:26:02,668: INFO: common: yaml file: params.yaml loaded successfully.]
[2024-04-10 22:26:02,668: INFO: common: yaml file: schema.yaml loaded successfully.]
[2024-04-10 22:26:02,668: INFO: common: created directory at: artifacts]
[2024-04-10 22:26:02,668: INFO: common: created directory at: artifacts/model_trainer]
[2024-04-10 22:26:03,178: INFO: 973666313: ]


In [48]:
%pwd
!python ./src/mlProject/pipeline/stage_04_model_trainer.py 

[2024-04-10 22:54:17,504: INFO: stage_04_model_trainer: >>>>>>>> stage Model Trainer stage started <<<<<<<<]
[2024-04-10 22:54:17,504: INFO: common: yaml file: config\config.yaml loaded successfully.]
[2024-04-10 22:54:17,504: INFO: common: yaml file: params.yaml loaded successfully.]
[2024-04-10 22:54:17,504: INFO: common: yaml file: schema.yaml loaded successfully.]
[2024-04-10 22:54:17,504: INFO: common: created directory at: artifacts]
[2024-04-10 22:54:17,504: INFO: common: created directory at: artifacts/model_trainer]
[2024-04-10 22:54:17,520: INFO: stage_04_model_trainer: >>>>>>>> stage Model Trainer stage completed <<<<<<<<

X===========X]


In [50]:
!python main.py

[2024-04-10 22:57:00,233: INFO: main: >>>>>>>> stage Data Ingestion stage started <<<<<<<<]
[2024-04-10 22:57:00,233: INFO: common: yaml file: config\config.yaml loaded successfully.]
[2024-04-10 22:57:00,233: INFO: common: yaml file: params.yaml loaded successfully.]
[2024-04-10 22:57:00,248: INFO: common: yaml file: schema.yaml loaded successfully.]
[2024-04-10 22:57:00,248: INFO: common: created directory at: artifacts]
[2024-04-10 22:57:00,248: INFO: common: created directory at: artifacts/data_ingestion]
[2024-04-10 22:57:01,334: INFO: data_ingestion: artifacts/data_ingestion/data.zip downloaded! with following info Connection: close
Content-Length: 26176
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "f751ec0917e15a3dc07c3094a49fb99713109dd8a9f29150bb5d187ec17facdf"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mo